In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import folium

In [2]:
# un country list
un = pd.read_csv("data/CountriesUN_20241120.csv", names=["UN","FIPS"])

In [3]:
un

,UN,FIPS
0,Afghanistan,AF
1,Albania,AL
2,Algeria,AG
3,Andorra,AN
4,Angola,AO
...,...,...
190,Yemen,YM
191,Zambia,ZA
192,Zimbabwe,ZI
193,The Holy See (Vatican City),VT


In [4]:
full = pd.read_csv("processed/full_calculation_20241117.csv") #'EPSG:3857'  # Web Mercator projection
full['geometry'] = full['geometry'].apply(wkt.loads)
full = gpd.GeoDataFrame(full, geometry='geometry', crs='EPSG:3857')

In [5]:
full

,FID,GMI_ADMIN,ADMIN_NAME,FIPS_CNTRY,CNTRY_NAME,STATUS,POP,POP_RANK,POP_CLASS,POP_SOURCE,...,Name1__sea,Name2__sea,Name3__sea,TYPE__sea,ISO_CC__sea,SHAPE_Length__sea,SHAPE_Area__sea,distance_sea,POP_rank_updated,city_type
0,1,BRA-MGR,Mato Grosso,BR,Brazil,Provincial capital,540814,3,"500,000 to 999,999",UN_Data_2010_2020,...,Atlantic Ocean,NaN,NaN,Ocean or Sea,NaN,1.064543,0.007229,1420.018591,3,inland
1,2,BRA-DFD,Distrito Federal,BR,Brazil,National and provincial capital,2481272,2,"1,000,000 to 4,999,999",UN_Data_2010_2020,...,Atlantic Ocean,NaN,NaN,Ocean or Sea,NaN,1.040116,0.007083,929.254581,2,inland
2,3,BRA-GOI,Goias,BR,Brazil,Provincial capital,1297154,2,"1,000,000 to 4,999,999",UN_Data_2010_2020,...,Atlantic Ocean,NaN,NaN,Ocean or Sea,NaN,0.422189,0.004244,906.965140,2,inland
3,4,BRA-MGD,Mato Grosso do Sul,BR,Brazil,Provincial capital,776242,3,"500,000 to 999,999",UN_Data_2010_2020,...,Atlantic Ocean,NaN,NaN,Ocean or Sea,NaN,1.064543,0.007229,883.599543,3,inland
4,5,PRY-AMM,Amambay,PA,Paraguay,Provincial capital,0,0,,,...,Atlantic Ocean,NaN,NaN,Ocean or Sea,NaN,1.064543,0.007229,852.927751,5,inland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535,2536,NZL-WST,Westland,NZ,New Zealand,Provincial capital,0,0,,,...,Tasman Sea,Pacific Ocean,NaN,Ocean or Sea,NaN,1.058717,0.057883,2.525252,7,coast
2536,2537,NZL-CNT,Canterbury,NZ,New Zealand,Provincial capital,388500,4,"250,000 to 499,999",UN_Data_2010_2020,...,Pacific Ocean,NaN,NaN,Ocean or Sea,NaN,1.007334,0.041327,8.413012,4,coast
2537,2538,NZL-OTG,Otago,NZ,New Zealand,Provincial capital,130700,5,"100,000 to 249,999",UN_Data_2010_2020,...,Pacific Ocean,NaN,NaN,Ocean or Sea,NaN,0.097013,0.000236,3.592056,5,coast
2538,2539,NZL-STH,Southland,NZ,New Zealand,Provincial capital,0,0,,,...,Tasman Sea,Pacific Ocean,NaN,Ocean or Sea,NaN,1.497316,0.038836,3.846671,7,coast


In [6]:
# manual change the city_type after manual check (see meeting notes with Eva Nov 19, 2024)
full.loc[full['CITY_NAME'] == 'Porto Novo', 'city_type'] = 'coast'
full.loc[full['CITY_NAME'] == 'Scoresbyund', 'city_type'] = 'coast'
full.loc[full['CITY_NAME'] == 'Patras', 'city_type'] = 'coast'
full.loc[full['CITY_NAME'] == 'Tripoli', 'city_type'] = 'coast'
full.loc[full['CITY_NAME'] == 'Bergen', 'city_type'] = 'coast'
full.loc[full['CITY_NAME'] == 'Riga', 'city_type'] = 'coast'

In [7]:
# if city_type of all rows with the same  FIPS_CNTRY is "coast",  give "all_coast" value 1, otherwise 0
full['all_coast'] = full.groupby('FIPS_CNTRY')['city_type'].transform(
    lambda x: 1 if all(val == "coast" for val in x) else 0
)

In [8]:
# if city_type of all rows with the same  FIPS_CNTRY is "inland",  give "all_inland" value 1, otherwise 0
full['all_inland'] = full.groupby('FIPS_CNTRY')['city_type'].transform(
    lambda x: 1 if all(val == "inland" for val in x) else 0
)

In [9]:
full['single_city_cntry'] = full.groupby('FIPS_CNTRY')['FIPS_CNTRY'].transform(
    lambda x: 1 if len(x) == 1 else 0
)

In [10]:
full['UN'] = full['FIPS_CNTRY'].apply(lambda x: 1 if x in un['FIPS'].values else 0)

In [11]:
full

,FID,GMI_ADMIN,ADMIN_NAME,FIPS_CNTRY,CNTRY_NAME,STATUS,POP,POP_RANK,POP_CLASS,POP_SOURCE,...,ISO_CC__sea,SHAPE_Length__sea,SHAPE_Area__sea,distance_sea,POP_rank_updated,city_type,all_coast,all_inland,single_city_cntry,UN
0,1,BRA-MGR,Mato Grosso,BR,Brazil,Provincial capital,540814,3,"500,000 to 999,999",UN_Data_2010_2020,...,NaN,1.064543,0.007229,1420.018591,3,inland,0,0,0,1
1,2,BRA-DFD,Distrito Federal,BR,Brazil,National and provincial capital,2481272,2,"1,000,000 to 4,999,999",UN_Data_2010_2020,...,NaN,1.040116,0.007083,929.254581,2,inland,0,0,0,1
2,3,BRA-GOI,Goias,BR,Brazil,Provincial capital,1297154,2,"1,000,000 to 4,999,999",UN_Data_2010_2020,...,NaN,0.422189,0.004244,906.965140,2,inland,0,0,0,1
3,4,BRA-MGD,Mato Grosso do Sul,BR,Brazil,Provincial capital,776242,3,"500,000 to 999,999",UN_Data_2010_2020,...,NaN,1.064543,0.007229,883.599543,3,inland,0,0,0,1
4,5,PRY-AMM,Amambay,PA,Paraguay,Provincial capital,0,0,,,...,NaN,1.064543,0.007229,852.927751,5,inland,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535,2536,NZL-WST,Westland,NZ,New Zealand,Provincial capital,0,0,,,...,NaN,1.058717,0.057883,2.525252,7,coast,0,0,0,1
2536,2537,NZL-CNT,Canterbury,NZ,New Zealand,Provincial capital,388500,4,"250,000 to 499,999",UN_Data_2010_2020,...,NaN,1.007334,0.041327,8.413012,4,coast,0,0,0,1
2537,2538,NZL-OTG,Otago,NZ,New Zealand,Provincial capital,130700,5,"100,000 to 249,999",UN_Data_2010_2020,...,NaN,0.097013,0.000236,3.592056,5,coast,0,0,0,1
2538,2539,NZL-STH,Southland,NZ,New Zealand,Provincial capital,0,0,,,...,NaN,1.497316,0.038836,3.846671,7,coast,0,0,0,1


In [12]:
len(full[full['UN'] == 1]['FIPS_CNTRY'].unique())

192

In [13]:
full[full['UN'] == 1]['FIPS_CNTRY'].unique()

array(['BR', 'PA', 'AR', 'PE', 'BL', 'WS', 'TN', 'CI', 'EC', 'CO', 'UY',
       'US', 'CA', 'MX', 'GT', 'CU', 'CS', 'PM', 'VE', 'NU', 'HO', 'ES',
       'BH', 'HA', 'DR', 'JM', 'BF', 'CD', 'NG', 'NI', 'CM', 'EK', 'GB',
       'CT', 'CF', 'CG', 'PL', 'TS', 'LY', 'IT', 'FR', 'MT', 'SM', 'BK',
       'AL', 'RB', 'MJ', 'MK', 'GR', 'TU', 'CY', 'EG', 'BU', 'RO', 'RS',
       'GG', 'SY', 'LE', 'UP', 'AM', 'IZ', 'SA', 'JO', 'IS', 'UG', 'OD',
       'SU', 'YM', 'ER', 'ET', 'DJ', 'SO', 'KE', 'NO', 'GM', 'DA', 'SZ',
       'LS', 'AU', 'SW', 'FI', 'EZ', 'HR', 'HU', 'SI', 'KZ', 'MG', 'IN',
       'PK', 'AF', 'KG', 'UZ', 'CH', 'TI', 'NP', 'BG', 'BT', 'CE', 'TH',
       'VM', 'LA', 'PS', 'RP', 'ID', 'JA', 'BM', 'MY', 'CB', 'SN', 'KN',
       'KS', 'BX', 'RM', 'FM', 'KR', 'AS', 'TT', 'PP', 'NZ', 'TD', 'GY',
       'NS', 'SC', 'AC', 'ST', 'DO', 'VC', 'BB', 'GJ', 'UK', 'IC', 'EI',
       'PO', 'SP', 'CV', 'MO', 'MR', 'AG', 'ML', 'PU', 'GV', 'SL', 'LI',
       'SG', 'GA', 'UV', 'IV', 'GH', 'TP', 'BN', 'T

In [ ]:
full.

In [14]:
result_rows = []

for country_code, group in full.groupby('FIPS_CNTRY'):
    
    # Find the capital city in the group
    capital_cities = group[group['capital']==1]
    
    if capital_cities.empty:
        continue  # No capital city in this country

    capital_city = capital_cities.iloc[0]

    capital_type = capital_city['city_type']

    # Determine the type of city to find
    if capital_type == 'inland':
    # Need to find a coast city
        candidate_cities = group[group['city_type'] != 'inland']
    else: # sea
    # Need to find a inland city
        candidate_cities = group[group['city_type'] == 'inland']
       
    # Exclude the capital city from candidate cities
    candidate_cities = candidate_cities[candidate_cities['CITY_NAME'] != capital_city['CITY_NAME']]

    if candidate_cities.empty:
        # No suitable second city, just keep the capital city
        result_rows.append(capital_city)
    else:
        # Find the city with the closest population to the capital
        candidate_cities = candidate_cities.copy()
        candidate_cities['pop_diff'] = (candidate_cities['POP_updated'] - capital_city['POP_updated']).abs()

        # Get the city with the smallest population difference
        second_city = candidate_cities.loc[candidate_cities['pop_diff'].idxmin()]

        # Append both capital city and second city
        result_rows.extend([capital_city, second_city])

# Create the result dataframe
result = gpd.GeoDataFrame(result_rows, crs="EPSG:3857")

In [28]:
result("processed/full_calculation_20241120.csv", index=False)

TypeError: 'GeoDataFrame' object is not callable

In [16]:
result.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 325 entries, 295 to 2311
Data columns (total 40 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   FID                      325 non-null    int64   
 1   GMI_ADMIN                325 non-null    object  
 2   ADMIN_NAME               325 non-null    object  
 3   FIPS_CNTRY               325 non-null    object  
 4   CNTRY_NAME               325 non-null    object  
 5   STATUS                   325 non-null    object  
 6   POP                      325 non-null    int64   
 7   POP_RANK                 325 non-null    int64   
 8   POP_CLASS                325 non-null    object  
 9   POP_SOURCE               325 non-null    object  
 10  geometry                 325 non-null    geometry
 11  CITY_NAME                325 non-null    object  
 12  POP_updated              325 non-null    float64 
 13  POP_updated_year         324 non-null    float64 
 14  capi

In [17]:
# first_batch = two cities selected in each country
result['first_batch'] = result.groupby('FIPS_CNTRY')['FIPS_CNTRY'].transform('count').apply(lambda x: 1 if x > 1 else 0)

In [18]:
result_s = result[["FID","CITY_NAME","CNTRY_NAME","FIPS_CNTRY","geometry","POP_updated","POP_rank_updated","capital","distance_sea","city_type","pop_diff","all_coast","all_inland","single_city_cntry","first_batch","UN"]]

In [19]:
result_s[result_s.first_batch==1]

,FID,CITY_NAME,CNTRY_NAME,FIPS_CNTRY,geometry,POP_updated,POP_rank_updated,capital,distance_sea,city_type,pop_diff,all_coast,all_inland,single_city_cntry,first_batch,UN
1909,1910,Algiers,Algeria,AG,POINT (339524.597 4408944.066),2364230.0,2,1,1.553963,coast,NaN,0,0,0,1,1
1926,1927,Batna,Algeria,AG,POINT (687064.156 4239835.492),1119791.0,2,0,171.612108,inland,1244439.0,0,0,0,1,1
630,631,Tirana,Albania,AL,POINT (2207688.627 5061435.007),418495.0,4,1,36.166383,inland,NaN,0,0,0,1,1
604,605,Durres,Albania,AL,POINT (2165609.868 5060694.538),113249.0,5,0,1.825464,coast,305246.0,0,0,0,1,1
2352,2353,Luanda,Angola,AO,POINT (1473313.952 -984837.475),6759313.0,1,1,0.295061,coast,NaN,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1285,1286,Nha Trang,Vietnam,VM,POINT (12156422.727 1373375.587),285788.0,4,0,0.000000,coast,27206.0,0,0,0,1,1
2332,2333,Windhoek,Namibia,WA,POINT (1901002.536 -2578894.465),486186.0,4,1,283.592946,inland,NaN,0,0,0,1,1
2328,2329,Swakopmund,Namibia,WA,POINT (1617249.385 -2592641.405),75921.0,6,0,0.576777,coast,410265.0,0,0,0,1,1
864,865,Sanaa,Yemen,YM,POINT (4921379.394 1730890.680),33908.0,7,1,149.616375,inland,NaN,0,0,0,1,1


In [20]:
# remove cities that are not in the UN list
result_s=result_s[result_s.UN==1]

In [21]:
#result.to_csv("processed/selected_cities_20241120.csv", index=False)

In [22]:
#result_s.to_csv("processed/selected_cities_s_20241120.csv", index=False)

In [27]:
full[full.FID==946].Name1__sea

945    Skagerrak
Name: Name1__sea, dtype: object